## Notebook 1
# Extraindo coordenadas de decretos e leis

Autores:
* Pedro C. de Siracusa
* André G. Coutinho
* Waira Saravia Machida
* Augusto Arcela
* Yan Felipe Soares

---

Neste estudo exploramos decretos e leis que embasam a criação de UCs, em âmbitos Federal e Municipal.
O objetivo é extrair as coordenadas que delimitam as UCs, de forma que possamos posteriormente projetar o polígono em um ambiente SIG. 
Os **principais desafios**:

* Os documentos estão no formato `.pdf`, não muito amigável para a extração de dados. Algumas bibliotecas auxiliam na extração de texto de arquivos `.pdf`, mas elas nem sempre funcionam como esperamos. Isso porque estes arquivos podem ter sido codificados de muitas formas diferentes, e nem sempre conseguimos fazer a extração.

* Os documentos não seguem um padrão estrutural, o que significa que nem sempre a informação que desejamos está no mesmo lugar ou se apresenta da mesma forma. Isso dificulta a extração inteiramente automatizada da informação nestes documentos.

* A delimitação das UCs é apresentada de forma descritiva nestes documentos. Em alguns poucos casos as coordenadas são organizadas em uma tabela, facilitando um pouco a extração.

Vamos explorar a **Lei No.11285** da Casa Civil da Presidencia da Republica, que determina os limites do Parque Nacional de Brasilia.
O documento está nomeado com o`l11285.pdf`, localizado dentro do diretório `data`, na raiz do projeto.

O primeiro passo foi extrair (manualmente) a parte do documento que descreve a delimitação da UC.
Armazenamos esta informação no arquivo `l11285_limits.txt`, para facilitar a leitura e extração de coordenadas do texto.

In [1]:
fpath = '../data/decretos/limits/l11285_limits_semquebra.txt'

Agora vamos abrir o arquivo, ler as linhas de texto e armazenar numa variável `lines`. Na célula abaixo usamos uma estrutura do *Python* que se chama [context manager](https://docs.python.org/3/reference/compound_stmts.html#with), para facilitar a abertura/fechamento de arquivos.

In [2]:
with open(fpath, encoding='ANSI') as f:
    lines =''.join( f.readlines() )

Vejamos os 1000 primeiros caracteres do texto:

In [3]:
lines[:1000]+' (...)'

'DELIMITAÇÕES: Partindo do vértice 1 de coordenadas N=8.274.617,0000 e E=175.882,0000, segue com o azimute 47º00\'47" e distância de 805,184 metros até o vértice 2 de coordenadas N=8.275.166,0000 e E=176.471,0000; daí, segue com o azimute 61º51\'30" e distância de 97,529 metros até o vértice 3 de coordenadas N=8.275.212,0000 e E=176.557,0000; daí, segue com o azimute 57º26\'22" e distância de 196,957 metros até o vértice 4 de coordenadas N=8.275.318,0000 e E=176.723,0000; daí, segue com o azimute 42º20\'45" e distância de 106,888 metros até o vértice 5 de coordenadas N=8.275.397,0000 e E=176.795,0000; daí, segue com o azimute 120º27\'56" e distância de 39,446 metros até o vértice 6 de coordenadas N=8.275.377,0000 e E=176.829,0000; daí, segue com o azimute 72º19\'18" e distância de 263,441 metros até o vértice 7 de coordenadas N=8.275.457,0000 e E=177.080,0000; daí, segue com o azimute 80º41\'38" e distância de 241,174 metros até o vértice 8 de coordenadas N=8.275.496,0000 e E=177.318,0

Neste caso, as coordenadas parecem estar descritas em UTM seguindo o padrão `E-XXX.XXX,XXXX e N-X.XXX.XXX,XXX`.
Para extrair o texto que segue este padrão vamos usar **expressões regulares**.

In [5]:
len(lines)

300083

---

## Regular Expressions

Expressões regulares servem para definirmos padrões de busca em um texto. Uma vez definido o padrão que desejamos buscar, o texto é percorrido tentando encontrar "casamentos", que podem então ser retornados. Usaremos expressões regulares para tentar extrair as coordenadas, assumindo que elas tenham sido inseridas em um padrão relativamente estável.

Alguns exemplos e guias de como se utilizar expressões regulares: 
1. https://medium.com/tech-tajawal/regular-expressions-the-last-guide-6800283ac034
1. https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285
1. http://www.rexegg.com/regex-quickstart.html#ref

O *Python* já fornece um módulo que nos permite trabalhar com expressões regulares de forma facilitada. O nome do módulo é `re`, e para usá-lo precisamos importá-lo. Veja [aqui](https://docs.python.org/3/howto/regex.html) como usar expressões regulares com o módulo `re`.

In [6]:
import re

#### Primeira tentativa

Abaixo vamos fazer uma primeira tentativa de expressão para capturar as coordenadas. Guardamos a expressão em uma variável de nome `expr`.

In [7]:
expr = '[EN]\= ?[0-9\.]+,[0-9]+'

Agora vamos usar a função `findall` para encontrar todos os *"matches"* para esta expressão. As coordenadas obtidas são armazenadas na variável `coords`. Para melhorar a visualização vou olhar apenas para as 10 primeiras coordenadas que foram obtidas.

In [8]:
coords = re.findall( expr , lines )

In [9]:
coords

['N=8.274.617,0000',
 'E=175.882,0000',
 'N=8.275.166,0000',
 'E=176.471,0000',
 'N=8.275.212,0000',
 'E=176.557,0000',
 'N=8.275.318,0000',
 'E=176.723,0000',
 'N=8.275.397,0000',
 'E=176.795,0000',
 'N=8.275.377,0000',
 'E=176.829,0000',
 'N=8.275.457,0000',
 'E=177.080,0000',
 'N=8.275.496,0000',
 'E=177.318,0000',
 'N=8.275.457,0000',
 'E=177.563,0000',
 'N=8.275.364,0000',
 'E=177.860,0000',
 'N=8.274.136,0000',
 'E=179.461,0000',
 'N=8.273.879,0000',
 'E=179.800,0000',
 'N=8.273.678,0000',
 'E=180.071,0000',
 'N=8.273.158,0000',
 'E=180.753,0000',
 'N=8.272.804,0000',
 'E=181.219,0000',
 'N=8.272.263,0000',
 'E=181.930,0000',
 'N=8.271.826,0000',
 'E=182.505,0000',
 'N=8.271.513,0000',
 'E=182.921,0000',
 'N=8.270.171,0000',
 'E=184.742,0000',
 'N=8.269.861,0000',
 'E=185.157,0000',
 'N=8.269.539,0000',
 'E=185.598,0000',
 'N=8.269.250,0000',
 'E=185.989,0000',
 'N=8.268.635,0000',
 'E=186.822,0000',
 'N=8.268.189,0000',
 'E=187.433,0000',
 'N=8.267.847,0000',
 'E=187.890,0000',


Quantas coordenadas foram obtidas ao todo?

In [10]:
len(coords)

4506

Um problema deste método que usamos acima é que fica um pouco difícil validar se as coordenadas realmente estão sendo extraídas em pares. Caso uma delas não tenha sido capturada, isso pode gerar uma deformação do polígono extraído.

#### Segunda tentativa

Vamos fazer uma segunda tentativa. Agora, queremos incluir também a palavra 'e' entre as coordenas, que parece sempre indicar um novo par de coordenadas no texto original.

In [11]:
expr = '[EN][\= ]+[0-9\.]+,[0-9]{1,4} e [EN][\= ]+[0-9\.]+,[0-9]{1,4}'

In [12]:
coords = re.findall( expr, lines )

In [13]:
coords # observando apenas as primeiras 10 entradas

['N=8.274.617,0000 e E=175.882,0000',
 'N=8.275.166,0000 e E=176.471,0000',
 'N=8.275.212,0000 e E=176.557,0000',
 'N=8.275.318,0000 e E=176.723,0000',
 'N=8.275.397,0000 e E=176.795,0000',
 'N=8.275.377,0000 e E=176.829,0000',
 'N=8.275.457,0000 e E=177.080,0000',
 'N=8.275.496,0000 e E=177.318,0000',
 'N=8.275.457,0000 e E=177.563,0000',
 'N=8.275.364,0000 e E=177.860,0000',
 'N=8.274.136,0000 e E=179.461,0000',
 'N=8.273.879,0000 e E=179.800,0000',
 'N=8.273.678,0000 e E=180.071,0000',
 'N=8.273.158,0000 e E=180.753,0000',
 'N=8.272.804,0000 e E=181.219,0000',
 'N=8.272.263,0000 e E=181.930,0000',
 'N=8.271.826,0000 e E=182.505,0000',
 'N=8.271.513,0000 e E=182.921,0000',
 'N=8.270.171,0000 e E=184.742,0000',
 'N=8.269.861,0000 e E=185.157,0000',
 'N=8.269.539,0000 e E=185.598,0000',
 'N=8.269.250,0000 e E=185.989,0000',
 'N=8.268.635,0000 e E=186.822,0000',
 'N=8.268.189,0000 e E=187.433,0000',
 'N=8.267.847,0000 e E=187.890,0000',
 'N=8.267.630,0000 e E=188.191,0000',
 'N=8.267.25

No texto original, existem no total 2361 vértices. Quantos vértices conseguimos extrair com o método acima?

In [14]:
len(coords)

2253

In [52]:
str1 = 'p'.join(coords)

In [53]:
str1

'N=8.274.617,0000 e E=175.882,0000pN=8.275.166,0000 e E=176.471,0000pN=8.275.212,0000 e E=176.557,0000pN=8.275.318,0000 e E=176.723,0000pN=8.275.397,0000 e E=176.795,0000pN=8.275.377,0000 e E=176.829,0000pN=8.275.457,0000 e E=177.080,0000pN=8.275.496,0000 e E=177.318,0000pN=8.275.457,0000 e E=177.563,0000pN=8.275.364,0000 e E=177.860,0000pN=8.274.136,0000 e E=179.461,0000pN=8.273.879,0000 e E=179.800,0000pN=8.273.678,0000 e E=180.071,0000pN=8.273.158,0000 e E=180.753,0000pN=8.272.804,0000 e E=181.219,0000pN=8.272.263,0000 e E=181.930,0000pN=8.271.826,0000 e E=182.505,0000pN=8.271.513,0000 e E=182.921,0000pN=8.270.171,0000 e E=184.742,0000pN=8.269.861,0000 e E=185.157,0000pN=8.269.539,0000 e E=185.598,0000pN=8.269.250,0000 e E=185.989,0000pN=8.268.635,0000 e E=186.822,0000pN=8.268.189,0000 e E=187.433,0000pN=8.267.847,0000 e E=187.890,0000pN=8.267.630,0000 e E=188.191,0000pN=8.267.254,0000 e E=188.697,0000pN=8.266.800,0000 e E=189.315,0000pN=8.266.386,0000 e E=189.874,0000pN=8.266.123,0

In [54]:
str2 = re.sub(r' ', '', str1)

In [55]:
str3 = re.sub(r'e', ' ', str2)

In [75]:
str3

'N=8.274.617,0000 E=175.882,0000pN=8.275.166,0000 E=176.471,0000pN=8.275.212,0000 E=176.557,0000pN=8.275.318,0000 E=176.723,0000pN=8.275.397,0000 E=176.795,0000pN=8.275.377,0000 E=176.829,0000pN=8.275.457,0000 E=177.080,0000pN=8.275.496,0000 E=177.318,0000pN=8.275.457,0000 E=177.563,0000pN=8.275.364,0000 E=177.860,0000pN=8.274.136,0000 E=179.461,0000pN=8.273.879,0000 E=179.800,0000pN=8.273.678,0000 E=180.071,0000pN=8.273.158,0000 E=180.753,0000pN=8.272.804,0000 E=181.219,0000pN=8.272.263,0000 E=181.930,0000pN=8.271.826,0000 E=182.505,0000pN=8.271.513,0000 E=182.921,0000pN=8.270.171,0000 E=184.742,0000pN=8.269.861,0000 E=185.157,0000pN=8.269.539,0000 E=185.598,0000pN=8.269.250,0000 E=185.989,0000pN=8.268.635,0000 E=186.822,0000pN=8.268.189,0000 E=187.433,0000pN=8.267.847,0000 E=187.890,0000pN=8.267.630,0000 E=188.191,0000pN=8.267.254,0000 E=188.697,0000pN=8.266.800,0000 E=189.315,0000pN=8.266.386,0000 E=189.874,0000pN=8.266.123,0000 E=190.234,0000pN=8.265.803,0000 E=190.680,0000pN=8.265

In [69]:
coords2 = str3.split('p')


In [76]:
coords2

['N=8.274.617,0000 E=175.882,0000',
 'N=8.275.166,0000 E=176.471,0000',
 'N=8.275.212,0000 E=176.557,0000',
 'N=8.275.318,0000 E=176.723,0000',
 'N=8.275.397,0000 E=176.795,0000',
 'N=8.275.377,0000 E=176.829,0000',
 'N=8.275.457,0000 E=177.080,0000',
 'N=8.275.496,0000 E=177.318,0000',
 'N=8.275.457,0000 E=177.563,0000',
 'N=8.275.364,0000 E=177.860,0000',
 'N=8.274.136,0000 E=179.461,0000',
 'N=8.273.879,0000 E=179.800,0000',
 'N=8.273.678,0000 E=180.071,0000',
 'N=8.273.158,0000 E=180.753,0000',
 'N=8.272.804,0000 E=181.219,0000',
 'N=8.272.263,0000 E=181.930,0000',
 'N=8.271.826,0000 E=182.505,0000',
 'N=8.271.513,0000 E=182.921,0000',
 'N=8.270.171,0000 E=184.742,0000',
 'N=8.269.861,0000 E=185.157,0000',
 'N=8.269.539,0000 E=185.598,0000',
 'N=8.269.250,0000 E=185.989,0000',
 'N=8.268.635,0000 E=186.822,0000',
 'N=8.268.189,0000 E=187.433,0000',
 'N=8.267.847,0000 E=187.890,0000',
 'N=8.267.630,0000 E=188.191,0000',
 'N=8.267.254,0000 E=188.697,0000',
 'N=8.266.800,0000 E=189.315

## Estruturando coordenadas

Agora que temos uma lista de strings armazenando coordenadas na forma `N=X.XXX.XXX,XXXX E=XXX.XXX,XXXX'`, precisamos extrair apenas os valores de coordenadas propriamente ditas.
Como cada coordenada é formada por um par de valores, vamos representá-las como tuplas. A função abaixo realiza esta tarefa:

In [72]:
def getCoords(coordsStr):
    coordsStrSplit = coordsStr.split(' ')
    return ( coordsStrSplit[0], coordsStrSplit[1] )

Sendo assim, podemos obter uma lista de pares ordenados, cada qual formando uma coordenada. Aplicaremos a função `getCoords` sobre cada elemento da lista na variável `coords`. Ao final, sobreescreveremos a variável `coords` para conter a nova lista de tuplas (em vez da antiga lista de strings).

In [73]:
gcoords = [ getCoords(c) for c in coords2 ]

In [74]:
gcoords

[('N=8.274.617,0000', 'E=175.882,0000'),
 ('N=8.275.166,0000', 'E=176.471,0000'),
 ('N=8.275.212,0000', 'E=176.557,0000'),
 ('N=8.275.318,0000', 'E=176.723,0000'),
 ('N=8.275.397,0000', 'E=176.795,0000'),
 ('N=8.275.377,0000', 'E=176.829,0000'),
 ('N=8.275.457,0000', 'E=177.080,0000'),
 ('N=8.275.496,0000', 'E=177.318,0000'),
 ('N=8.275.457,0000', 'E=177.563,0000'),
 ('N=8.275.364,0000', 'E=177.860,0000'),
 ('N=8.274.136,0000', 'E=179.461,0000'),
 ('N=8.273.879,0000', 'E=179.800,0000'),
 ('N=8.273.678,0000', 'E=180.071,0000'),
 ('N=8.273.158,0000', 'E=180.753,0000'),
 ('N=8.272.804,0000', 'E=181.219,0000'),
 ('N=8.272.263,0000', 'E=181.930,0000'),
 ('N=8.271.826,0000', 'E=182.505,0000'),
 ('N=8.271.513,0000', 'E=182.921,0000'),
 ('N=8.270.171,0000', 'E=184.742,0000'),
 ('N=8.269.861,0000', 'E=185.157,0000'),
 ('N=8.269.539,0000', 'E=185.598,0000'),
 ('N=8.269.250,0000', 'E=185.989,0000'),
 ('N=8.268.635,0000', 'E=186.822,0000'),
 ('N=8.268.189,0000', 'E=187.433,0000'),
 ('N=8.267.847,0

In [83]:
len(gcoords)

2253

> nota: a estrutura acima é nativa do *Python*, e é conhecida como "compreensão de listas" (**list comprehension**). Ela permite a construção de listas de forma bastante rápida, usando expressões 'for'.

Agora precisamos validar e corrigir as coordenadas na lista `coords`.

---

## Validando as coordenadas

In [80]:
fixDecimal = lambda x: x.replace('.','').replace(',','.')

In [82]:
[ (fixDecimal(N), fixDecimal(E)) for N,E in coords2 ]

ValueError: too many values to unpack (expected 2)